# Cryptocurrency Investment Strategy

### Previously, I was involved in the cryptocurrency field, in part due to the craze that caught my attention. I do believe that the value of these currencies are probably not justified at this point in time. However, I believe there’s still an opportunity to make some profits with these currencies by trading them.

### Hence, I will try to cover trading strategies as and when I can to show what works and what doesn't.

#### Strategy:
>  Moving Average Crossover Trading Strategy Backtest (MACD)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv("../input/all_currencies.csv")
df.head()

In [ ]:
# Remove all NaN values in Market Cap column
df2=df.dropna(subset=['Market Cap'])
df2.head()

In [ ]:
df2.reset_index(drop = True, inplace = True)
df2.head()

In [ ]:
# Top 20 cryptocurrencies in market
Top_20_coins=df2.sort_values(['Date','Market Cap'], ascending = False).head(20)

In [ ]:
Top_20_coins.head()

In [ ]:
len(Top_20_coins['Symbol'].unique())

# Overall Market Capitalization of Cryptocurrencies

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

trace1 = go.Bar(
    x = Top_20_coins["Symbol"].tolist(),
    y = Top_20_coins["Market Cap"].tolist(),
    name='Country Overall',
    marker=dict(
        color='rgba(55, 128, 191, 0.7)',
        line=dict(
            color='rgba(55, 128, 191, 1.0)',
            width=2,
        )
    )
)

layout = go.Layout(
    barmode='stack',
    title = 'Market capitalization across cryptos',
    titlefont=dict(size=25),
    width=850,
    height=500,
    paper_bgcolor='rgb(244, 238, 225)',
    plot_bgcolor='rgb(244, 238, 225)',
    yaxis = dict(
        title= 'Market Cap',
        anchor = 'x',
        rangemode='tozero'
    ),
    xaxis = dict(title= 'Coin'),
    yaxis2=dict(
        title='Country Value [M€]',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right',
        anchor = 'x',
        rangemode = 'tozero',
        dtick = 200
    ),
    #legend=dict(x=-.1, y=1.2)
    legend=dict(x=0.05, y=0.05)
)
data = [trace1]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [ ]:
df2.columns

# Bitcoin Price History

In [ ]:
df3 = df2[df2['Symbol']=='BTC']
df4 = df3.set_index('Date')
df4.head()

In [ ]:
df5 = df4['Close']

In [ ]:
df5.head()

# Visualization of Bitcoin Price History

In [ ]:
from matplotlib import pyplot
series = df5
print(series.head())
series.plot()
pyplot.show()

# Ethereum Investment Strategy (by Moving Averages)

We will have 3 basic states/rules:

1) Buy Signal (go long) – the 20d moving average is for the first time X points above the 40d tend.

2) Park in Cash – no position.

3) Sell Signal (go short) – the 20d moving average is for the first time X points below the 40d trend.

> The first step in creating these signals is to add a new column to the DataFrame which is just the difference between the two moving averages:

In [ ]:
# 20 days Moving average
df2['20d'] = np.round(df2['Close'].rolling(window=20).mean(),2)
# 40 days Moving average
df2['40d'] = np.round(df2['Close'].rolling(window=40).mean(),2)
df2.tail()

In [ ]:
import matplotlib.pyplot as plt
df3 = df2[df2['Symbol']=='ETH']
df3['DateTime'] = pd.to_datetime(df3['Date'])
plt.xticks(rotation=45)
plt.plot(df3['DateTime'],df3[['Close','20d','40d']])
plt.legend(['Close','20d','40d'])


In [ ]:
df3['20-40'] = df3['20d'] - df3['40d']
X= 20
df3['Stance'] = np.where(df3['20-40'] > X, 1, 0)
df3['Stance'] = np.where(df3['20-40'] < X, -1, df3['Stance'])
df3['Stance'].value_counts()

## Interpretation of result

### The above results shows that during the time period we have chosen to backtest, on 855 trading dates the 20d moving average lies more than 20 points below the 40d moving average, and on 177 the 20d moving average lies more than 20 points above the 40d moving average.

In [ ]:
df3['Stance'].plot(lw=1.5,ylim=[-1.1,1.1])

### Everything is now in place to test our investment strategy based upon the signals we have generated. In this instance we assume for simplicity that ETH  can be bought or sold directly and that there are no transaction costs. 

### So in this model, our investor is either long the market, short the market or flat – this allows us to work with market returns and simply multiply the day’s market return by -1 if he is short, 1 if he is long and 0 if he is flat the previous day.

In [ ]:
df4_for_later_use = df3
df3.tail()

In [ ]:
df3['Market Returns'] = np.log(df3['Close'] / df3['Close'].shift(1))
df3['Strategy'] = df3['Market Returns'] * df3['Stance'].shift(1)

In [ ]:
plt.plot(df3['DateTime'],df3[['Market Returns','Strategy']].cumsum())
plt.xticks(rotation=45)
plt.legend(['Market Returns','Strategy'])

## Analysis of results

### So we can see that although the strategy seems to perform rather well during market downturns, it doesn’t do so well during market rallies or when it is just trending upwards.

### Over the test period it barely outperforms a simple buy and hold strategy, hardly enough to call it a “successful” strategy. THIS STRATEGY IS USELESS (jk please read on...)
### Maybe we shouldn't conclude so soon. There was a huge rally due to the cryptocurrency craze in 2017. Let's examine the strategy on Feb - May  2018 after the bitcoin bubble burst to see if the analysis paints a different picture.

# Let us try zooming into the time horizon Feb - May 2018

In [ ]:
df5 = df4_for_later_use[(df4_for_later_use['Date'] > '2018-02-01') & (df4_for_later_use['Date'] < '2018-05-01')]
df5.head()

In [ ]:
plt.plot(df5['DateTime'],df5[['Market Returns','Strategy']].cumsum())
plt.xticks(rotation=45)
plt.legend(['Market Returns','Strategy'])

## Final Verdict
### There we go, the trading strategy actually shows huge potential when the markets are less bullish, huge gains would have been made within the last couple of months if this was actually implemented!


### And we are done! A moving average cross over strategy backtested in Python! :)